In [16]:
import pandas as pd
from xgboost import XGBClassifier
import time
from sklearn import metrics
import shap
import numpy as np
from xgboost import plot_importance

In [17]:
df = pd.read_csv('soccerDB5.csv')

In [18]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [19]:
df.drop(columns=['home_player_1',
 'home_player_2',
 'home_player_3',
 'home_player_4',
 'home_player_5',
 'home_player_6',
 'home_player_7',
 'home_player_8',
 'home_player_9',
 'home_player_10',
 'home_player_11',
 'away_player_1',
 'away_player_2',
 'away_player_3',
 'away_player_4',
 'away_player_5',
 'away_player_6',
 'away_player_7',
 'away_player_8',
 'away_player_9',
 'away_player_10',
 'away_player_11',
 'sub_home_player_1',
 'sub_home_player_2',
 'sub_home_player_3',
 'sub_home_player_4',
 'sub_home_player_5',
 'sub_home_player_6',
 'sub_home_player_7',
 'sub_home_player_8',
 'sub_home_player_9',
 'sub_home_player_10',
 'sub_home_player_11',
 'sub_home_player_12',
 'sub_away_player_1',
 'sub_away_player_2',
 'sub_away_player_3',
 'sub_away_player_4',
 'sub_away_player_5',
 'sub_away_player_6',
 'sub_away_player_7',
 'sub_away_player_8',
 'sub_away_player_9',
 'sub_away_player_10',
 'sub_away_player_11',
 'sub_away_player_12',
 'city',
 'referee',
 'stadium'], inplace=True)

In [20]:
df.columns.values

array(['a_Ataques', 'a_Ataques perigosos', 'a_Cartões amarelos',
       'a_Cartões vermelhos', 'a_chutes_a_gol', 'a_Chutes ao ferro',
       'a_Chutes bloqueados', 'a_Chutes fora', 'a_Escanteios', 'a_Faltas',
       'a_Gols', 'a_Impedimentos', 'a_Laterais', 'a_Pênaltis',
       'a_Segundo cartão amarelo', 'a_Substituições', 'a_Tiros de meta',
       'a_Tiros-livres', 'at_current_classification',
       'at_current_draws_away', 'at_current_draws_global',
       'at_current_draws_home', 'at_current_loss_away',
       'at_current_loss_global', 'at_current_loss_home',
       'at_current_wins_away', 'at_current_wins_global',
       'at_current_wins_home', 'at_games_without_draw_away',
       'at_games_without_draw_global', 'at_games_without_draw_home',
       'at_games_without_lose_away', 'at_games_without_lose_global',
       'at_games_without_lose_home', 'at_games_without_win_away',
       'at_games_without_win_global', 'at_games_without_win_home',
       'away_team', 'away_team_score', '

In [21]:
categorical_features_no_players=['away_team',
 'home_team']

In [22]:
df.datetime = pd.to_datetime(df.datetime, format='%Y-%m-%d')

In [23]:
def to_integer(dt_time):
    return dt_time.dt.day + 100*dt_time.dt.month + 10000*dt_time.dt.year

In [24]:
%%time
df.datetime = to_integer(df.datetime)

CPU times: user 5.69 ms, sys: 0 ns, total: 5.69 ms
Wall time: 6.69 ms


In [25]:
# def runModel(df, categorical_features):
#     t0 = time.time()    
#     data = df.drop(columns=['home_team_score','away_team_score', 'a_Gols', 'h_Gols', 'result'])
#     label = df['result']
#     data = pd.get_dummies(data, dummy_na=True, columns=categorical_features)
#     print(data.shape)
#     a = round(len(df)*.8)
#     X_train, X_test, y_train, y_test = data[0:a], data[a:], label[0:a], label[a:]
#     params = {
#         'max_depth': 10,
#         'objective': 'multi:softmax',  # error evaluation for multiclass training
#         'num_class': 3,   
#     }
#     model = XGBClassifier(param=params,nthread=-1)
#     model.fit(X_train, y_train)
#     pred = model.predict(X_test)
#     cm = metrics.confusion_matrix(y_test, pred)
#     print ('run time: ', time.time()-t0,'s')
#     print("Accuracy:",metrics.accuracy_score(y_test, pred))
#     print (cm)
#     ax = plot_importance(model, max_num_features=1000)
#     fig = ax.figure
#     fig.set_size_inches(18,100)

In [26]:
from sklearn.model_selection import GridSearchCV

In [37]:
def runModel(df, categorical_features):
    t0 = time.time()    
    data = df.drop(columns=['home_team_score','away_team_score', 'a_Gols', 'h_Gols', 'result'])
    label = df['result']
    data = pd.get_dummies(data, dummy_na=True, columns=categorical_features)
    print(data.shape)
    a = round(len(df)*.8)
    X_train, X_test, y_train, y_test = data[0:a], data[a:], label[0:a], label[a:]
    gbm_param_grid = {
        'learning_rate':[0.01, 0.1, 0.5 ,0.9 , 1],
        'n_estimators':[200],
        'colsample_bytree':[0.01, 0.1, 0.5, 1],
        'max_depth': [6, 10, 15, 20],
        'gamma': [0, 1, 2, 3],
        'lambda': [1, 3, 5, 7]
    }
#     params = {
#         'objective': 'multi:softmax',  # error evaluation for multiclass training
#         'num_class': 3,   
#     }
    model = XGBClassifier(nthread=-1)
    grid_mse = GridSearchCV(estimator=model, param_grid=gbm_param_grid, scoring='accuracy', cv=4, verbose=1)
    grid_mse.fit(X_train, y_train)
    pred = grid_mse.predict(X_test)
    cm = metrics.confusion_matrix(y_test, pred)
    print ('run time: ', time.time()-t0,'s')
    print("Accuracy:",metrics.accuracy_score(y_test, pred))
    print (cm)
    ax = plot_importance(model, max_num_features=1000)
    fig = ax.figure
    fig.set_size_inches(18,100)

In [38]:
%%time
model = runModel(df, categorical_features_no_players)

(2271, 144)
Fitting 4 folds for each of 1280 candidates, totalling 5120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5120 out of 5120 | elapsed: 211.7min finished
/home/renan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


run time:  12703.292353391647 s
Accuracy: 0.5374449339207048
[[  3  10 119]
 [  5  10  70]
 [  2   4 231]]


XGBoostError: need to call fit or load_model beforehand

In [12]:
df.shape

(2271, 87)